In [5]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import time
model = Word2Vec.load("/Users/chunyu/Downloads/kaggle/bag_words/part2test/300features_40minwords_10context")
start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
print type(model.wv)
word_vectors = model.wv.vectors
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

<class 'gensim.models.word2vec.Word2Vec'>
Time taken for K Means clustering:  662.00917387 seconds.


In [6]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'language', u'speakers', u'languages', u'words']

Cluster 1
[u'ronnie', u'marsha', u'miner', u'formerly', u'picard', u'evan', u'snyder', u'janice', u'hearst', u'lucinda', u'wellington', u'logan']

Cluster 2
[u'topical', u'pertinent', u'scarce', u'timely', u'restricted', u'prevalent']

Cluster 3
[u'arrow', u'axe', u'analogy', u'outbreak', u'alligator', u'eclipse', u'autopsy', u'octopus', u'explosion', u'egg', u'android', u'electrical', u'umbrella', u'unseen', u'aquarium', u'iceberg', u'entity', u'astronaut', u'insect', u'avalanche', u'accidental', u'ax', u'eagle', u'electric']

Cluster 4
[u'seducing', u'urges']

Cluster 5
[u'channels', u'surfing', u'flipping', u'browsing', u'screens']

Cluster 6
[u'lacked', u'absent', u'lacks', u'lacking', u'devoid']

Cluster 7
[u'pistol', u'groin', u'shotgun', u'gun', u'knife', u'jackhammer', u'corpse', u'grenade', u'cord', u'bullet', u'goat', u'machete']

Cluster 8
[u'discussions', u'transformations', u'metaphor', u'messages', u'meanings'

In [16]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,'lxml').get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [18]:
import pandas as pd
import numpy as np

train = pd.read_csv("/Users/chunyu/Downloads/kaggle/bag_words/all/labeledTrainData.tsv", \
                    header=0,delimiter="\t",quoting=3)
test = pd.read_csv( "/Users/chunyu/Downloads/kaggle/bag_words/all/testData.tsv", header=0, delimiter="\t", quoting=3 )

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))
print len(clean_train_reviews)
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))
counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

25000


In [19]:
from sklearn.ensemble import RandomForestClassifier
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "/Users/chunyu/Downloads/kaggle/bag_words/part2test/BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
